# User-based Collaborative Filtering

Core idea:
"If Alice and Bob have rated movies similarly in the past, then we can recommend to Alice the movies that Bob liked (but Alice hasn’t seen yet)."

How it works:

Find users similar to the target user (e.g., Alice).

Look at what those similar users liked.

Recommend those movies to Alice.

Example:
If Alice and Bob both liked Inception and The Matrix, and Bob also liked Interstellar, then Interstellar might be recommended to Alice.

In [4]:
import pandas as pd
import os
base_path = r'C:\Users\Sara\Documents\DS Masters\Semester 2\Solution Engineering\Python\movies-database\ml-25m'

In [1]:
import pandas as pd
import os

# Base path to your local data
base_path = r'C:\Users\Sara\Documents\python proj'

# Load base datasets
movies = pd.read_csv(os.path.join(base_path, 'movies.csv'))
links = pd.read_csv(os.path.join(base_path, 'links.csv'))
ratings = pd.read_csv(os.path.join(base_path, 'ratings.csv'))

# Sample a small subset (10k ratings)
ratings_sample = ratings.sample(n=10000, random_state=42)
movie_ids_sample = ratings_sample['movieId'].unique()

# Filter other datasets based on sampled movieIds
movies_sample = movies[movies['movieId'].isin(movie_ids_sample)]
links_sample = links[links['movieId'].isin(movie_ids_sample)]

# Optionally add tags, genome if desired
tags = pd.read_csv(os.path.join(base_path, 'tags.csv'))
tags_sample = tags[tags['movieId'].isin(movie_ids_sample)]

genome_scores = pd.read_csv(os.path.join(base_path, 'genome-scores.csv'))
genome_tags = pd.read_csv(os.path.join(base_path, 'genome-tags.csv'))
genome = pd.merge(genome_scores, genome_tags, on='tagId', how='left')
genome_sample = genome[genome['movieId'].isin(movie_ids_sample)]

# Merge datasets
df = ratings_sample \
    .merge(movies_sample, on='movieId', how='left') \
    .merge(links_sample, on='movieId', how='left') \
    .merge(tags_sample, on=['userId', 'movieId'], how='left') \
    .merge(genome_sample, on='movieId', how='left')

# Save to file
df.to_csv(os.path.join(base_path, 'movies_sampled_merged.csv'), index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Sara\\Documents\\python proj\\movies.csv'

In [5]:
movies_merged  = pd.read_csv(os.path.join(base_path, 'movies_sampled_merged.csv'))


C:\Users\Sara\AppData\Local\Temp\ipykernel_10220\2553455272.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_merged  = pd.read_csv(os.path.join(base_path, 'movies_sampled_merged.csv'))


In [6]:
movies_merged.head()

,userId,movieId,rating,timestamp_x,title,genres,imdbId,tmdbId,tag_x,timestamp_y,tagId,relevance,tag_y
0,99476,104374,3.5,1467897440,About Time (2013),Drama|Fantasy|Romance,2194499,122906.0,NaN,NaN,1.0,0.02975,007
1,99476,104374,3.5,1467897440,About Time (2013),Drama|Fantasy|Romance,2194499,122906.0,NaN,NaN,2.0,0.02875,007 (series)
2,99476,104374,3.5,1467897440,About Time (2013),Drama|Fantasy|Romance,2194499,122906.0,NaN,NaN,3.0,0.04750,18th century
3,99476,104374,3.5,1467897440,About Time (2013),Drama|Fantasy|Romance,2194499,122906.0,NaN,NaN,4.0,0.06125,1920s
4,99476,104374,3.5,1467897440,About Time (2013),Drama|Fantasy|Romance,2194499,122906.0,NaN,NaN,5.0,0.04150,1930s


In [7]:
len(movies_merged)

11354603

In [18]:
movies_merged.movieId.nunique()

3658

User-based Collaborative Filtering

In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Step 1: Create a user-item ratings matrix
user_item_matrix = movies_merged.pivot_table(index='userId', columns='movieId', values='rating')

# Step 2: Normalize (center) ratings by subtracting user mean
user_means = user_item_matrix.mean(axis=1)
user_item_centered = user_item_matrix.sub(user_means, axis=0)

# Step 3: Fill NaNs with 0 for cosine similarity
user_item_filled = user_item_centered.fillna(0)

# Step 4: Compute user-user similarity matrix
user_sim = cosine_similarity(user_item_filled)
user_sim_df = pd.DataFrame(user_sim, index=user_item_filled.index, columns=user_item_filled.index)

In [15]:
target_user = 99476
target_movie = 104374

# Find users who rated movie 104374
users_who_rated = user_item_matrix[target_movie].dropna()

# Similarities of those users to target user
similarities = user_sim_df.loc[target_user, users_who_rated.index]

# Ratings by similar users for the movie
ratings = users_who_rated

# Weighted average
predicted_rating = (similarities * ratings).sum() / similarities.sum()
print(f"Predicted rating for user {target_user} and movie {target_movie}: {predicted_rating:.2f}")

Predicted rating for user 99476 and movie 104374: nan


C:\Users\Sara\AppData\Local\Temp\ipykernel_13524\3262982872.py:14: RuntimeWarning: invalid value encountered in scalar divide
  predicted_rating = (similarities * ratings).sum() / similarities.sum()


In [16]:
def predict_rating(user_id, movie_id, user_item_matrix, user_sim_df, fallback='user_mean'):
    if movie_id not in user_item_matrix.columns or user_id not in user_item_matrix.index:
        return None  # movie or user not in data

    # Users who rated this movie
    users_rated = user_item_matrix[movie_id].dropna()

    if users_rated.empty:
        return None  # no one rated this movie

    # Similarities between target user and others who rated the movie
    sims = user_sim_df.loc[user_id, users_rated.index]
    ratings = users_rated

    # Compute weighted average
    weighted_sum = (sims * ratings).sum()
    sim_sum = sims.sum()

    if sim_sum == 0:
        if fallback == 'user_mean':
            return user_item_matrix.loc[user_id].mean()
        elif fallback == 'global_mean':
            return user_item_matrix.stack().mean()
        else:
            return None

    return weighted_sum / sim_sum

In [17]:
pred = predict_rating(99476, 104374, user_item_matrix, user_sim_df)
print(f"Predicted rating: {pred:.2f}" if pred else "Prediction unavailable")

Predicted rating: 3.50


In [5]:
# Load ratings and movie info
movies = pd.read_csv(os.path.join(base_path, 'movies.csv'))
ratings = pd.read_csv(os.path.join(base_path, 'ratings.csv'))

# Merge to get movie titles (optional, for display)
ratings_merged = ratings.merge(movies[['movieId', 'title']], on='movieId', how='left')

# Drop missing values (just in case)
ratings_merged.dropna(subset=['userId', 'movieId', 'rating'], inplace=True)



In [6]:
# Check the structure
ratings_merged.head()

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,Pulp Fiction (1994)
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994)
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993)
3,1,665,5.0,1147878820,Underground (1995)
4,1,899,3.5,1147868510,Singin' in the Rain (1952)


In [8]:
len(ratings_merged)

25000095

Option 1: Filter to “Active” Users and/or “Popular” Movies

In [9]:
# Keep users with at least 500 ratings
user_counts = ratings_merged['userId'].value_counts()
active_users = user_counts[user_counts >= 500].index

# Keep movies with at least 1000 ratings
movie_counts = ratings_merged['movieId'].value_counts()
popular_movies = movie_counts[movie_counts >= 1000].index

# Filter the DataFrame
filtered = ratings_merged[
    ratings_merged['userId'].isin(active_users) &
    ratings_merged['movieId'].isin(popular_movies)
]

In [10]:
len(filtered)

7127698

In [20]:
# Assuming 'filtered' is your cleaned dataset with userId, movieId, rating, and title
user_item_matrix = filtered.pivot_table(index='userId', columns='movieId', values='rating')

# Mean-center and fill NaNs with 0 (for cosine similarity)
user_item_centered = user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0).fillna(0)

# Save for later use in prediction
user_ids = user_item_centered.index.tolist()
movie_ids = user_item_centered.columns.tolist()


In [23]:
user_item_centered 

movieId,1,2,3,4,5,6,7,8,9,10,...,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,
3,0.302050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
12,0.663313,-1.336687,-1.336687,0.000000,0.000000,0.000000,-0.336687,0.0,0.0,-0.336687,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
72,0.000000,-1.371912,0.000000,0.000000,0.000000,0.628088,0.628088,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
80,0.000000,-1.561441,0.000000,0.000000,0.000000,2.438559,0.000000,0.0,0.0,2.438559,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
120,0.748718,0.000000,0.000000,0.000000,0.000000,0.000000,-0.251282,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,0.406332,-0.093668,0.000000,-0.093668,-1.093668,0.000000,0.000000,0.0,0.0,-0.093668,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
162495,0.091423,0.091423,0.591423,0.000000,-0.908577,0.000000,0.000000,0.0,0.0,1.091423,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
162508,1.294654,0.000000,0.000000,0.000000,0.000000,-0.205346,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [24]:
from annoy import AnnoyIndex

# Number of movies (features per user)
f = len(movie_ids)
annoy_index = AnnoyIndex(f, metric='angular')  # angular ≈ cosine

user_id_map = {}               # userId -> index in annoy
reverse_user_id_map = {}       # index in annoy -> userId

for i, user_id in enumerate(user_ids):
    vector = user_item_centered.loc[user_id].values
    annoy_index.add_item(i, vector)
    user_id_map[user_id] = i
    reverse_user_id_map[i] = user_id

# Build with 10 trees (balance speed/accuracy)
annoy_index.build(n_trees=10)

True

In [28]:
def predict_rating_annoy(user_id, movie_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, k=30, min_raters=3):
    if user_id not in user_id_map or movie_id not in user_item_matrix.columns:
        return None

    u_idx = user_id_map[user_id]
    neighbors = annoy_index.get_nns_by_item(u_idx, k, include_distances=False)
    neighbor_ids = [reverse_user_id_map[i] for i in neighbors if reverse_user_id_map[i] != user_id]

    ratings = []
    for neighbor_id in neighbor_ids:
        rating = user_item_matrix.loc[neighbor_id, movie_id]
        if not np.isnan(rating):
            ratings.append(rating)

    if len(ratings) < min_raters:
        return user_item_matrix.loc[user_id].mean()  # fallback

    return np.mean(ratings)

In [29]:
def recommend_top_n_annoy(user_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, movies_df, n=5, k_neighbors=30):
    if user_id not in user_item_matrix.index:
        return []

    unrated_movies = user_item_matrix.columns[user_item_matrix.loc[user_id].isna()]
    predictions = []

    for movie_id in unrated_movies:
        pred = predict_rating_annoy(user_id, movie_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, k=k_neighbors)
        if pred is not None:
            predictions.append((movie_id, pred))

    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    return [(mid, movies_df.loc[movies_df.movieId == mid, 'title'].values[0], score) for mid, score in top_n]

In [30]:
example_user = user_item_matrix.index[0]

top_recs = recommend_top_n_annoy(
    user_id=example_user,
    user_item_matrix=user_item_matrix,
    annoy_index=annoy_index,
    user_id_map=user_id_map,
    reverse_user_id_map=reverse_user_id_map,
    movies_df=filtered,
    n=5
)

# Print recommendations
print(f"\nTop 5 Recommendations for User {example_user}:")
for movie_id, title, score in top_recs:
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")


Top 5 Recommendations for User 3:
In the Mood For Love (Fa yeung nin wa) (2000) (Movie ID: 4144) — Predicted Rating: 4.83
Persona (1966) (Movie ID: 7327) — Predicted Rating: 4.75
On the Waterfront (1954) (Movie ID: 1945) — Predicted Rating: 4.67
Fanny and Alexander (Fanny och Alexander) (1982) (Movie ID: 2068) — Predicted Rating: 4.62
12 Angry Men (1957) (Movie ID: 1203) — Predicted Rating: 4.53


Evaluation

In [31]:
from sklearn.model_selection import train_test_split

# Split the original filtered ratings
train_df, test_df = train_test_split(filtered, test_size=0.2, random_state=42)

In [32]:
user_item_matrix_train = train_df.pivot_table(index='userId', columns='movieId', values='rating')
user_item_centered_train = user_item_matrix_train.sub(user_item_matrix_train.mean(axis=1), axis=0).fillna(0)

user_ids = user_item_centered_train.index.tolist()
movie_ids = user_item_centered_train.columns.tolist()

# Annoy index
f = len(movie_ids)
annoy_index = AnnoyIndex(f, metric='angular')

user_id_map = {}
reverse_user_id_map = {}

for i, user_id in enumerate(user_ids):
    annoy_index.add_item(i, user_item_centered_train.loc[user_id].values)
    user_id_map[user_id] = i
    reverse_user_id_map[i] = user_id

annoy_index.build(n_trees=10)

True

In [34]:
from sklearn.metrics import root_mean_squared_error
true_ratings = []
predicted_ratings = []

for _, row in test_df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    true_rating = row['rating']
    
    pred = predict_rating_annoy(user_id, movie_id, user_item_matrix_train, annoy_index, user_id_map, reverse_user_id_map, k=30)
    
    if pred is not None:
        true_ratings.append(true_rating)
        predicted_ratings.append(pred)

rmse = root_mean_squared_error(true_ratings, predicted_ratings)
print(f"RMSE: {rmse:.4f}")

KeyboardInterrupt: 

Option 2: Use a Sparse Matrix Instead of Full pivot_table

In [18]:
from scipy.sparse import csr_matrix

# Reindex for compatibility
user_mapper = {uid: i for i, uid in enumerate(ratings_merged['userId'].unique())}
movie_mapper = {mid: i for i, mid in enumerate(ratings_merged['movieId'].unique())}

ratings_merged['user_index'] = ratings_merged['userId'].map(user_mapper)
ratings_merged['movie_index'] = ratings_merged['movieId'].map(movie_mapper)

# Create sparse matrix
rows = ratings_merged['user_index']
cols = ratings_merged['movie_index']
data = ratings_merged['rating']

sparse_user_item = csr_matrix((data, (rows, cols)))

SCIKIT-SURPRISE

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict

# 2. Load your filtered DataFrame
# Assumes 'filtered' contains: userId, movieId, rating, title
ratings = filtered[['userId', 'movieId', 'rating']].copy()

# 3. Prepare data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings, reader)

# 4. Split into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 5. Build the User-Based CF model
sim_options = {
    'name': 'cosine',
    'user_based': True  # Set to False for item-based
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# 6. Evaluate on test set
predictions = algo.test(testset)
rmse(predictions)

In [2]:
import numpy
print(numpy.__version__)

1.26.4
